In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Create Neural Network

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    #1 input image channel channel, filter size of 6, 3x3 square convolution kernel
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 3) 
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 3)

    #an affine operation y = Mx + b. Simply, Dense layer of Fully Connected layer
    #in_features = 6x6 (image_size) * 16 (output channels from previous layers)
    self.fc1 = nn.Linear(in_features = 16 * 6 * 6, out_features = 120) #out_features ~= nodes in Tensorflow
    self.fc2 = nn.Linear(in_features = 120, out_features = 84)
    self.fc3 = nn.Linear(in_features = 84, out_features = 10)

  def forward(self, x):
      #Max pooling over
      x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size = (2,2))
      #If the size if a square you can only specify a signle number
      x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size = (2,2))
      x = torch.flatten(x, start_dim = 1, end_dim = -1) #Torch supports mini-batch only
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
  def num_flat_features(self, x):
    size = x.size()[1:] #all dimension except the batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features 
net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Get trainable parameters

In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


#### Random inputs

In [0]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 1.4819e-04,  2.0977e-02, -1.0035e-01,  9.8798e-02, -5.3361e-02,
          1.6321e-01,  5.7239e-02,  2.2307e-02, -1.1789e-01, -1.1450e-01]],
       grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1,10))

#### Loss Function

In [0]:
output = net(input)
target = torch.randn(10) #dummy target, for example
target = target.expand_as(output)

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)
print(loss.backward)

tensor(1.0326, grad_fn=<MseLossBackward>)
<bound method Tensor.backward of tensor(1.0326, grad_fn=<MseLossBackward>)>


In [0]:
#print loss in backward_direction
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [0]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad before backward')
print(net.conv1.grad)

conv1.bias.grad before backward
None
conv1.bias.grad before backward
tensor([-0.0153,  0.0094,  0.0141,  0.0046,  0.0079, -0.0006])


#### Update weights

In [0]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [0]:
import torch.optim as optim
#create optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

#in training loop
optimizer.zero_grad() #zero the gradient buffers
outut = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() #Does the update